<a href="https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/016-Spectral-Indices-From-Awesome-Spectral-Indices-List.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectral Indices From the [Awesome Spectral Indices for GEE](https://github.com/davemlz/awesome-ee-spectral-indices)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Conda-forge: [https://anaconda.org/conda-forge/eemont](https://anaconda.org/conda-forge/eemont)
- Documentation: [https://eemont.readthedocs.io/](https://eemont.readthedocs.io/)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/tutorials](https://github.com/davemlz/eemont/tree/master/tutorials)

## Let's start!

If required, please uncomment:

In [ ]:
#!pip install eemont
#!pip install geemap

Import the required packges.

In [ ]:
import ee, eemont, geemap
import geemap.colormaps as cm

Authenticate and Initialize Earth Engine and geemap.

In [ ]:
Map = geemap.Map()

Let's define a point of interest:

In [ ]:
poi = ee.Geometry.PointFromQuery("El Cairo, Egypt",user_agent = "eemont-tutorial-016")

Let's preprocess the S2 dataset:

In [ ]:
S2 = (ee.ImageCollection("COPERNICUS/S2_SR")
      .filterBounds(poi)
      .filterDate("2020-01-01","2021-01-01")
      .maskClouds()
      .scaleAndOffset())

## New method for computing Spectral Indices

In order to compute Spectral Indices from the Awesome List of Spectral Indices, please use the `spectralIndices` method for ee.Image and ee.ImageCollection classes (This method will eventually replace the `index` method):

In [ ]:
S2offline = S2.spectralIndices(["NDVI","kNDVI","NDWI","SeLI"]).median()

The `spectralIndices` method will look into the Awesome List of Spectral Indices that comes with the eemont package (offline). If the list is outdated (the local list is updated with every new version or with the dev installation), you can use the `online` argument to use the most recent list directly from the [Awesome Spectral Indices for GEE repositoy](https://github.com/davemlz/awesome-ee-spectral-indices):

In [ ]:
S2online = S2.spectralIndices(["NDVI","kNDVI","NDWI","SeLI"],online = True).median()

The `spectralIndices` has new arguments for some of the new indices that were added to the list:

- cexp (float, default = 1.16) – Exponent used for OCVI.
- nexp (float, default = 2.0) – Exponent used for GDVI.
- alpha (float, default = 0.1) – Weighting coefficient used for WDRVI.
- slope (float, default = 1.0) – Soil line slope.
- intercept (float, default = 0.0) – Soil line intercept.

Check the [eemont API Reference](https://eemont.readthedocs.io/en/latest/classes/index.html) and the [Awesome Spectral indices Documentation](https://awesome-ee-spectral-indices.readthedocs.io/en/latest/) for more info.

Now, let's visualize our indices!

In [ ]:
vegetation_vis = {
    "min": 0,
    "max": 1,
    "palette": cm.palettes.ndvi
}

water_vis = {
    "min": 0,
    "max": 1,
    "palette": cm.palettes.ndwi
}

Visualize everything with `geemap`:

In [ ]:
Map.addLayer(S2online.select("NDVI"),vegetation_vis,"NDVI")
Map.addLayer(S2online.select("kNDVI"),vegetation_vis,"kNDVI")
Map.addLayer(S2online.select("SeLI"),vegetation_vis,"SeLI")
Map.addLayer(S2online.select("NDWI"),water_vis,"NDWI")
Map.centerObject(poi)
Map